# Capstone Project - Battle of Neighborhoods (week 2)

# Applied Data Science Capstone

## Introduction to Business Problem

Opening a new French Restaurant in Houston,Texas.
The objective of this report is to determine the best possible location to open a new French Restaurant in Houston. it will be based on the different localities of the city, already established French restaurants in various geographical location and ease of accessibility by maximum number of people so that the revenue from the latest venture can be maximized.

## Data

In this project I will use data from :
Geopy - For getting the co-ordinated of different locations.
Foursquare API - To get the list of venues and their details around a given location.



## Methodology

Getting the co-ordinates of the target city.
Getting the list of neighborhoods and their co-ordinates.
Exploring the most visited venues in the target localities.
Clustering the localities.
Analyzing the clusters formed.
Collecting information about the type of restaurants already present in a locality.
Creating a machine learning model based on data acquired.
Using the model to predict the locality best suited for upcoming Italian Resataurant.

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [2]:
pip install geopy

     |████████████████████████████████| 112kB 7.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.1
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    attrs-20.3.0               |     pyhd3deb0d_0          41 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36he6145b8_1001         347 KB  conda-forge
    ca-certificates-2020.11.8  |       ha878542_0         145 KB  conda-forge
    certifi-2020.11.8          |   py36h5fab9bb

In [4]:
address = 'Houston, Texas'
geolocator = Nominatim(user_agent="my_user_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Houston are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Houston are 29.7589382, -95.3676974.


Houston neighborhood_latitude = 29.7589382
Houston neighborhood_longitude = -95.3676974

Dial FourSquare to find venues around current residence in Houston Texas

In [5]:
CLIENT_ID = '1CVX0P22PGNHR2GYDNEIYSSE5UZHVOGVTVADWQXJ5NF5BKJ1' # your Foursquare ID
CLIENT_SECRET = 'NU3QOJRDYYVJN0125XOZDZFQGLYVYSC5R5IEGBHXHL0SFFRO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1CVX0P22PGNHR2GYDNEIYSSE5UZHVOGVTVADWQXJ5NF5BKJ1
CLIENT_SECRET:NU3QOJRDYYVJN0125XOZDZFQGLYVYSC5R5IEGBHXHL0SFFRO


In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

In [16]:
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1CVX0P22PGNHR2GYDNEIYSSE5UZHVOGVTVADWQXJ5NF5BKJ1&client_secret=NU3QOJRDYYVJN0125XOZDZFQGLYVYSC5R5IEGBHXHL0SFFRO&v=20180604&ll=29.7589382,-95.3676974&radius=1000&limit=100'

In [18]:
results = requests.get(url).json()
#results

In [19]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc85e007c1f474e477697a7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Houston',
  'headerFullLocation': 'Downtown Houston, Houston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 184,
  'suggestedBounds': {'ne': {'lat': 29.767938209000008,
    'lng': -95.35734953273267},
   'sw': {'lat': 29.74993819099999, 'lng': -95.37804526726732}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bbb8ef0e452952192d954a4',
       'name': "Jason's Deli",
       'location': {'address': '901 McKinney St',
        'lat': 29.75746388891063,
        'lng': -95.36554282298964,
        'labeledL

function that extracts the category of the venue - borrow from the Foursquare lab

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


(100, 4)

In [23]:
SGnearby_venues.head(10)

,name,categories,lat,lng
0,Jason's Deli,Food Truck,29.757464,-95.365543
1,Hobby Center for the Performing Arts,Performing Arts Venue,29.761526,-95.369376
2,Mango Tree Thai Bistro,Thai Restaurant,29.758251,-95.365387
3,Becks Prime,Burger Joint,29.758185,-95.366172
4,Alley Theatre,Theater,29.761671,-95.365313
5,Perbacco,Italian Restaurant,29.760257,-95.364773
6,Bombay Pizza Co.,Pizza Place,29.757700,-95.364586
7,Jones Hall,Concert Hall,29.761240,-95.365301
8,Sam Houston Park,Park,29.759876,-95.371211
9,City Hall Farmer's Market,Farmers Market,29.759164,-95.368446


Map of Houston with venues in Neighborhood - for reference

In [2]:
# create map of Houston Texas using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg

NameError: name 'folium' is not defined

MANHATTAN NEIGHBORHOODS - DATA AND MAPPING

In [15]:
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans